# PSM Worksheet 1: $\delta^{18}O$ of foraminifera
In this worksheet, we'll build a simple forward model for the oxygen isotope composition of planktic foraminifera in the ocean. This model will be a **forward model**, meaning it predicts $\delta^{18}O_{calcite}$ from temperature and $\delta^{18}O_{seawater}$. 

We're going to run this model for modern Atlanic Ocean data to give **pseudo-proxy** $\delta^{18}O_{calcite}$ values for a given foram species, and then compare the output to real "core-top" $\delta^{18}O$ data.  




## Background
_This practical is inspired by the work of [Schmidt, G.A., 1999](https://doi.org/10.1029/1999PA900025)_ and [Malevich et al., 2019](https://doi.org/10.1029/2019PA003576)

### The $\delta^{18}O$ of foraminifera
The oxygen isotope composition of the calcite shells of foraminfera ($\delta^{18}O_{calcite}$) is potentially the most important proxy in paleoceanography. Early measurements of $\delta^{18}O_{calcite}$ on planktic foraminifera samples in deep-sea sediment cores gave us our first insight into the rythmic glacial cycles of the Pleistocene [Emiliani et al., 1955](), revealing periodic fluctiations in ocean temperature and global ice volume. 

$\delta^{18}O_{calcite}$ is a function of the **ambient $\delta^{18}O$ of seawater ($\delta^{18}O_{sw}$)** and the water temperature in which the foraminfera formed its shell. Each species of foraminifera will have distinct seasonal and depth preferences in the ocean. For example, one species might thrive in the upper 50m during July and August, while another might prefer the 100-200m range during April and May. 

To model a foraminifera shells' $\delta^{18}O_{calcite}$, we need to know the following things:
1. The depth range in which the foraminifera grew it's shell (it's **depth habitat**).
2. The month/season that the foraminifera grew it's shell (it's **seasonal preference**).
3. An equation relating water temperature and $\delta^{18}O_{sw}$ to $\delta^{18}O_{calcite}$.

With this information, we want our model to **predict $\delta^{18}O_{calcite}$ for a given species of planktic foraminifera in the modern ocean**. 


### Calibration equations
The relationship between $\delta^{18}O_{calcite}$, Temperature (T) and $\delta^{18}O_{sw}$ has been estimated from laboratory experiments (synthesizing inorganic calcite in a lab) and foraminifera culturing (growing a foram at a known temperature and d18Osw and then measuring the shell). These equations are usually presented in the form:

\begin{align}
T = a + b(\delta^{18}O_{calcite} - \delta^{18}O_{sw} - 0.27) + c(\delta^{18}O_{calcite} - \delta^{18}O_{sw} - 0.27)^2 
\end{align}

In this equation, a, b, and c are our coefficients (we can use different coefficients from different studies if we want), and we're subtracting 0.27 from $\delta^{18}O_{sw}$ to convert between VSMOW and VPDB (see box below).

Since we want to predict d18Ocalcite, we need to rearange like so:

\begin{align}
\delta^{18}O_{calcite} = \delta^{18}O_{sw} + \frac{b+\sqrt{b^2-4c(a-T)}}{2c}
\end{align}

To write this as a python function, I would suggest the following steps:
1. convert $\delta^{18}O_{sw}$ into VPDB (i.e. subtract 0.27‰)
2. calculate the _discriminant_ (the part of the equation under the square root symbol)... this should be `discriminant = b**2 - 4*c*(a - T)`
3. put the discriminant in the equation and solve for $\delta^{18}O_{calcite}$... something like `d18O_calcite = d18O_sw_VPDB + (-b + np.sqrt(discriminant)) / (2 * c)`

```{admonition} Note on delta notation and units 
:class: tip, dropdown
### A note on oxygen isotope scales: VSMOW versus VPDB
Oxygen isotope measurements are usually given in "delta" notation, with units of "per mille" (a.k.a per throusand, denoted by ‰). This is because the absolute abundance of minor isotopes like $^{18}O$ can't be measured accurately, so they are instead compared to a known standard material. You can think of this as _the difference of this sample from this standard x 1000_... or: 

\begin{align}
\delta_{sample} = \frac{R_{sample} - R_{standard}}{R_{standard}} x 1000
\end{align}


 To make life more confusing for MPhil students, oxygen isotopes measured on seawater are usually reported relative to a standard material known as Vienna Standard Mean Ocean Water (VSMOW), while carbonate oxygen isotope isotopes are usually reported relative to a carbonate standard known as Vienna PeeDee Belemnite (VPDB).
```

For this practical, you should use one of the following sets of coefficients:

| Name | a    | b    | c    | ref                   | method                  |
|------|------|------|------|-----------------------|-------------------------|
| KO97 | 16.1 | 4.64 | 0.05 | Kim and O'Neil (1997) | Inorganic calcite (lab) |
| EL83 | 17.0 | 4.52 | 0.03 | Erez and Luz (1983)   | Cultured forams         |
| M02  | 14.3 | 4.34 | 0.09 | Mulitza et al., 2002  | Field calibration       |


----
## Task 1: Predict calcite $\delta^{18}O$
For task 1, **write a function that takes `T`, `d18Osw`, and the equation coefficients `a`, `b`, and `c` as inputs, and returns $\delta^{18}O_{calcite}$.**  

Your function should take the form:

```
def equilibrium_calcite(d18Osw, T, a, b, c):
    # your code here
    return d18O_calcite
```

Once you're finished, test your code with the example parameters given in the woksheet notebook. 

----
## Task 2: Sample real ocean data.
Next up, we need to load some real ocean data to start building our PSM around. 

We need data for:
- temperature
- d18O of seawater

We want this data to be gridded (with lat, lon, and depth dimensions), and available as monthly climatologies (i.e. average values for Jan, Feb... etc.)

We're going to use data from [this study by Charlotte Breitkreuz et al., in 2018](https://doi.org/10.1029/2018JC014300). I accessed the original data [here](https://doi.pangaea.de/10.1594/PANGAEA.889922) and cleaned it up a little to make life easier. You can see exactly what I did in the `data/clean_data.py` file on the github page for this session. The full data reference is:
>Supplement to: Breitkreuz, C et al. (2018): A dynamical reconstruction of the global monthly-mean oxygen isotopic composition of seawater. Journal of Geophysical Research: Oceans, 123(10), 7206-7219, https://doi.org/10.1029/2018JC014300 

For the first part of task 2 (task 2.1), you need to load the data into an xarray dataframe using `xr.load_dataset(path)` where `path` is the location of the dataset relative to your working directory. Call the dataframe `ocean_data`. 


### Task 2.2: Sample the data
Next, we need to write a function to sample the dataset at a given latitude, longitude, depth, and month. To do this, [use the `dataframe.sel()` syntax](https://docs.xarray.dev/en/latest/user-guide/indexing.html). 

Your function should take the form:
```
def sample_data(lat,lon,depth,month):
    # your code here
    # make sure you return a dataframe containing BOTH temperature and d18Osw
    return data_i
```

Once you're finished, test your code with the example parameters given in the woksheet notebook. 

----
## Task 3: Create a function for foraminifera ecology
Next, we need to know the ecological preferences of some key foraminifera species. That way, we can lookup which month and which water depth a given species likely forms it shell in at each lat-lon grid cell. 

To do this, we're going to use an ecological model which provides gridded abundance data for three species of foraminifera by lat, lon, depth, and month. This is a modified file from the PLAFOM model created by [Kretschmer et al., 2018](https://doi.org/10.5194/bg-15-4405-2018)

For example, lets say we want to predict $\delta^{18}O_{calcite}$ for _G. bulloides_ at 50ºN and -20ºE. We ned to first find the nearest grid cell in our ecological dataset (PLAFOM) and return the depth and month for which that species is most abundant. The result of this search is shown in {numref}example_gbull_abundance.  


```{figure} img/example_Gbull_abundance.png
---
width: 400px
name: example_gbull_abundance
---
Abundance pattern for _G. bulloides_ in the foram ecology model "PLAFOM", published by [Kretschmer et al., 2018](https://doi.org/10.5194/bg-15-4405-2018). The darker blues refer to higher concentration and the cells refer to depths and months.
```

```{margin}
You can read more about foraminifera depth preferences [in this study by Rebotim et al.,](http://www.biogeosciences.net/14/827/2017/). Knowing how far below the surface a foram formed it's shell has big implications for interpreting the paleotemperature. Similarly, knowing the seasonal preference for a foram will influence how we interpret our proxy records. A temperature record from a foram that prefers late Autumn is going to tell us something different to a temperature record from a spring foram! You can read more about foram seasonal preferences [in this study by Jonkers et al., 2017](https://doi.org/10.5194/cp-13-573-2017)

```

Your function should look like this:
```
def find_preference(lat,lon,species):
    # your code here
    return depth_pref, month_pref
```

Once you're finished, test your code with the example parameters given in the woksheet notebook. 

----
## Task 4: Bring it together
You should now have 3 functions written:
- `equilibrium_calcite`
- `sample_data`
- `find_preference`

This should be all we need to build a simple PSM, whereby we can input a species and a location and return the predicted $\delta^{18}O_{calcite}$. Your task now is to create a new function that does exactly this, by calling these individual functions within.

```{admonition} A useful helper function 
:class: tip, dropdown

You'll no doubt realise that the `sample_data` requires month as an integer (where 0 = Jan, 1 = Feb, etc.), but the output from `find_preference` is a string ('Jan', 'Feb', etc.). You may have found a clever way to deal with this, but this helper function might save you some time:

def monthToNum(shortMonth):
    return {
            'jan': 1,
            'feb': 2,
            'mar': 3,
            'apr': 4,
            'may': 5,
            'jun': 6,
            'jul': 7,
            'aug': 8,
            'sep': 9, 
            'oct': 10,
            'nov': 11,
            'dec': 12
    }[shortMonth]
    
```